### Criar um banco caso não tenha, efetuando o test via dapper e criar conexao.

> Database.cs

> CreateDatabase(string connectionWithoutDataBase, string nameDatabase)

onde será aberta uma conexão, verificar se há um banco da dados ativo, caso contrario, vai ser criando um novo dataBase mediate o paramentro. 
   

### .Domain

>RepositoryExtension.cs

Crio uma extensão utilizando o serviço Extensions.Configuration, com objetivo de proteger o acesso a connectionString que estava exposto no program.cs da aplicação sendo usada por ConfigurationManager

Faço referencia ao IConfiguration, passando no paramentro ( configurationManager) que injeta o GetConnectionString

Extensions.Configuration é uma biblioteca para .NET que fornece suporte para a leitura de dados de configuração de uma variedade de fontes


### .infrastructure / Migrations / Versions

As versões será responsavel para criar 1 ou mais tabelas. Cada versão vai criar tabelas relacionadas conforme as mudanças na aplicação. 

> VersionData0000001.cs


Na VersionData0000001 preciso herdar de Migration do FluentMigrator que implementa o UP e DOWN. 

UP - para criar todo esquema das tabelas. 

> BaseVersion.cs

Para evitar escrever o codigo para uma tabela que precise utilizar date ou ID que estão como padrão, crio uma schema BASE. 

Na classe BaseVersion eu crio uma função que chamo do Flunt o ICreateTableColumnOptionOrWithColumnSyntax, retornando o table. Que vai criar tabela padrão para todo esquema. 

            .WithColumn("Id").AsInt64().PrimaryKey().Identity()
            .WithColumn("CreateDate").AsDateTime().NotNullable();
 
O que ele faz? Recebe a tabela do versionData, insere as colunas default e retorna a propria tabela. 

> São colunas especificadas de cada tabela.

            .WithColumn("Name").AsString(100).NotNullable()
            .WithColumn("Email").AsString(100).NotNullable()
            .WithColumn("Password").AsString(2000).NotNullable()
            .WithColumn("Phone").AsString(14).NotNullable();

            

Identifico a classe version como um migration, passando a flag 

> [Migration(1, "Criar tabela usuario")]

Melhorando o codigo: como o migration flag espera um numero, crio uma classe do tipo ENUM, que digo 1 == Criar tabela usuario

```Csharp
public enum SpecificVersionNumber
{
    CreateUserTable = 1
}
```

Na flag eu passo o metodo com o tipo [Migration((long)] e o ENUM.


MigrationExtension

Tem como proposito subir e organizar as versões de forma automatica. Passando como parametro o IApplicationBuilder que faz parte do Build(), e Do WebApplication que herda de IapplicationBuilder.

//Vai ser executado o app.MigrateDataBase(); que faz parte do MigrationExtensio. 

### Bootmigration

Bootmigration = Que fara o papel de verificar o que tem em cada VersionData0000001.cs e subir o que está contido no metodo UP()

AddRepository contem um Service que é um IServiceCollection. 


```csharp
//Metodo que verificar versão de migration atraves de metodo Fluent. 
private static void AddFluentMigrator(IServiceCollection services, IConfiguration configurationManager)
{
        services.AddFluentMigratorCore().ConfigureRunner(c =>
        c.AddSqlServer()
        .WithGlobalConnectionString(configurationManager.GetConnectionCompletParamenter())
        .ScanIn(Assembly.Load("Codeblox.DigitalMenu.Infrastructure")).For.All()); 
}
```